# Active Scans Report (BMC Discovery)

This notebook fetches discovery run activity from a BMC Discovery appliance using the REST API and loads the results into pandas for inspection and export.

In [ ]:
# TODO: Import from Excel

## Requirements

We use `requests` for HTTP, `pandas` for tabular data, and `PyYAML` to read configuration. If they are not installed, uncomment the install cell below.

In [ ]:
# %pip install -q tideway pandas pyyaml

import pandas as pd
import yaml
from pathlib import Path
from typing import Any, Dict, Optional
import xml.etree.ElementTree as ET
import tideway


In [ ]:
APPLIANCE_NAME: Optional[str] = None
APPLIANCE_INDEX: int = 0


## Select Appliance (optional)

If your `config.yaml` defines multiple appliances under the `appliances:` list, set `APPLIANCE_NAME` to one of their names (recommended) or set `APPLIANCE_INDEX` to pick by position. Leave both as-is to default to the first appliance.

In [ ]:
from pathlib import Path

def load_config_params(start: Path, appliance_name: Optional[str] = None, appliance_index: int = 0) -> Dict[str, Any]:
    def _find_repo_root(start_path: Path) -> Path:
        for p in [start_path] + list(start_path.parents):
            if (p / "config.yaml").exists():
                return p
        return start_path.parent

    repo_root = _find_repo_root(start)
    config_path = repo_root / "config.yaml"
    with open(config_path, "r") as fh:
        cfg = yaml.safe_load(fh) or {}

    apps = cfg.get("appliances") or []
    selected = None
    if isinstance(apps, list) and apps:
        if appliance_name:
            selected = next((a for a in apps if a.get("name") == appliance_name), None)
            if selected is None:
                raise ValueError(f"No appliance named '{appliance_name}' in config.yaml")
        else:
            try:
                selected = apps[int(appliance_index)]
            except Exception:
                selected = apps[0]

    target = ((selected or {}).get("target") or cfg.get("target") or "").strip()
    if not target:
        raise ValueError('config.yaml missing "target"')

    token = (((selected or {}).get("token") or cfg.get("token") or "").strip())
    token_file = (selected or {}).get("token_file") or cfg.get("token_file") or cfg.get("f_token")
    if not token and token_file:
        tf_path = Path(token_file)
        if not tf_path.is_absolute():
            tf_path = repo_root / tf_path
        with open(tf_path, "r") as tf:
            token = tf.read().strip()
    if not token:
        raise ValueError("API token not found in config.yaml (token or token_file)")

    api_version = str((selected or {}).get("api_version") or cfg.get("api_version") or "v1.14")
    verify_ssl = bool((selected or {}).get("verify_ssl", cfg.get("verify_ssl", True)))

    sanitized = target.replace('.', '_').replace(':', '_').replace('/', '_')
    output_dir = repo_root / f"output_{sanitized}"
    output_dir.mkdir(parents=True, exist_ok=True)

    return {
        "repo_root": repo_root,
        "config_path": config_path,
        "cfg": cfg,
        "selected": selected,
        "target": target,
        "token": token,
        "api_version": api_version,
        "verify_ssl": verify_ssl,
        "output_dir": output_dir,
    }


In [ ]:
def init_appliance(appliance_name: Optional[str] = "prod") -> Dict[str, Any]:
    params = load_config_params(Path.cwd(), appliance_name=appliance_name)
    target = params["target"]
    api_number = params["api_version"].lstrip('v')

    print('Base Host     :', target)
    print('API Version   :', api_number)
    print('Verify SSL    :', params["verify_ssl"])
    print('Output folder :', params["output_dir"])

    app = tideway.appliance(target, params["token"], api_version=api_number, ssl_verify=params["verify_ssl"])
    twsearch = app.data()
    twdisco = app.discovery()

    try:
        about = app.api_about
        print('Appliance reachable:', getattr(about, 'status_code', 'ok'))
    except Exception as exc:
        print('Warning: failed to contact appliance /api/about:', exc)

    return {
        "params": params,
        "target": target,
        "app": app,
        "search": twsearch,
        "disco": twdisco,
        "api_version": api_number,
        "output_dir": params["output_dir"],
        "name": (params["selected"] or {}).get("name") or (appliance_name or target),
    }


In [ ]:
instances: list[Dict[str, Any]] = []

def _attempt_init(name: Optional[str]):
    label = name if name else 'default'
    try:
        print(f"Initialise {label.capitalize()}:")
        inst = init_appliance(name)
        instances.append(inst)
    except Exception as exc:
        print(f"Skipping {label}: {exc}")

_attempt_init('prod')
_attempt_init('dev')

if not instances:
    _attempt_init(None)

if not instances:
    raise RuntimeError('No appliances could be initialised from config.yaml')

In [ ]:
def show_runs(inst: Dict[str, Any]) -> pd.DataFrame:
    disco = inst["disco"]
    getter = getattr(disco, "get_discovery_runs", None)
    if getter is None:
        message = "Discovery client missing get_discovery_runs"
        print(f"Request failed for {inst['name']}: {message}")
        return pd.DataFrame([{"DiscoveryInstance": inst["target"], "Message": message}])

    try:
        response = getter() if callable(getter) else getter
    except Exception as exc:
        message = f"Request failed: {exc}"
        print(f"Request failed for {inst['name']}: {exc}")
        return pd.DataFrame([{"DiscoveryInstance": inst["target"], "Message": message}])

    status_code = getattr(response, "status_code", 200)
    if status_code != 200:
        message = f"Request failed: {status_code}"
        print(f"Request failed for {inst['name']}: {status_code}")
        return pd.DataFrame([{"DiscoveryInstance": inst["target"], "Message": message}])

    if hasattr(response, "json"):
        try:
            data = response.json()
        except Exception as exc:
            message = f"Invalid response payload: {exc}"
            print(f"No discovery runs returned for {inst['name']}: {exc}")
            return pd.DataFrame([{"DiscoveryInstance": inst["target"], "Message": message}])
    else:
        data = response

    if not data:
        message = "No rows returned"
        print(f"No discovery runs returned for {inst['name']}")
        return pd.DataFrame([{"DiscoveryInstance": inst["target"], "Message": message}])

    df = pd.json_normalize(data)
    if df.empty:
        message = "No rows returned"
        print(f"No discovery runs returned for {inst['name']}")
        return pd.DataFrame([{"DiscoveryInstance": inst["target"], "Message": message}])

    df.insert(0, "DiscoveryInstance", inst["target"])
    return df


## Fetch discovery runs

Call the Discovery API endpoint that lists discovery runs. We normalize the JSON into a pandas DataFrame.

In [ ]:
all_runs: list[pd.DataFrame] = []
runs_by_instance: Dict[str, pd.DataFrame] = {}

## Inspect common fields

Show a few relevant columns such as labels, timing and counts when present.

In [ ]:
# Combine all dataframes into one
if all_runs:
    combined_df = pd.concat(all_runs, ignore_index=True)
else:
    combined_df = pd.DataFrame()

display(combined_df.head())

## Save to CSV (optional)

Persist the full dataset to the project output directory (`output_<target>`).


In [ ]:
for inst in instances:
    df = show_runs(inst)

    name = inst["name"] or inst["target"]
    runs_by_instance[name] = df
    all_runs.append(df)

    output_path = inst["output_dir"] / "active_scans.csv"
    df.to_csv(output_path, index=False)
    print(f"Saved runs for {name} to {output_path}")

if not all_runs:
    print("No discovery runs available to display.")


---
### Notes
- If your appliance uses a self-signed certificate, set `VERIFY_SSL = False`.
- If the appliance exposes a different API version, update `API_VERSION`.
- You can further transform the dataset with `pandas.json_normalize` or additional joins if needed.